# Gather companies and bios for those companies
### Big picture:
For companies gathered from http://www.nasdaq.com/screening/company-list.aspx, use stock ticker (and occasionally exchange name) to lookup SIC code, along with company profiles on Bloomberg, Google Finance, and Reuters. 

## Basic setup

In [6]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import json
import matplotlib.pyplot as plt
%matplotlib inline

#### Make BS and requests play together a little more nicely

In [10]:
def get_soup(url, to = None):
    r = requests.get(url,timeout = to)
    soup = bs(r.text,'lxml')
    return soup

## Import stock data

#### and add information about exchanges (here, labeled "Markets" -- probably not the technical term)

In [7]:
nasdaq = pd.read_csv("../data/SIC/nasdaq/nasdaq_from_nasdaq.csv")
nyse = pd.read_csv("../data/SIC/nasdaq/nyse_from_nasdaq.csv")
nasdaq['Market'] = 'NASDAQ'
nyse['Market'] = 'NYSE'
both = pd.concat([nasdaq, nyse])

## Functions that gather info based on stock symbol

#### Lookup SIC code on EDGAR 
(adapted from https://mktstk.com/2015/03/03/sic-lookup-by-stock-symbol/)


In [11]:
def lookup_sic_code(symbol):
    url = "https://www.sec.gov/cgi-bin/browse-edgar?CIK=" + symbol.upper() + "&owner=exclude&action=getcompany&Find=Search"
    try:
        soup = get_soup(url)
        return int(soup.find('acronym', {'title' : 'Standard Industrial Code'}).findNextSibling("a").contents[0])
    except:
        return None

#### Lookup profile on Bloomberg
Example: https://www.bloomberg.com/quote/NFLX:US

In [12]:
def lookup_bloomberg_profile(symbol):
    url = "https://www.bloomberg.com/quote/" + symbol.upper() + ":US"
    try:
        soup = get_soup(url)
        return str(soup.find('div',{'class' : 'profile__description'}).contents[0])
    except:
        return None

#### Lookup profile on Google Finance
Example: https://www.google.com/finance?q=NASDAQ%3ANFLX

(needs info about exchange)

In [13]:
def lookup_google_profile(symbol, exchange):
    url = "https://www.google.com/finance?q=" + exchange + "%3A" + symbol.upper()
    try: 
        soup = get_soup(url)
        return str(soup.find('div', {'class' : 'companySummary'}).contents[0])
    except:
        return None
    return description

#### Lookup profile on Reuters
Example: http://www.reuters.com/finance/stocks/companyProfile?rpc=66&symbol=NFLX.O

In [18]:
# seems to not need exchange...can figure it out mostly
# though it's possible I'm mistaken about this
def lookup_reuters_profile(symbol):
    url = "http://www.reuters.com/finance/stocks/companyProfile?symbol=" + symbol.upper()
    try: 
        soup = get_soup(url)
        paragraphs = soup.find('div', {'id' : 'companyNews'}).findAll('p')
        return str('\n'.join([(lambda x: x.contents[0] if len(x.contents)>0 else '')(p) for p in paragraphs]))
    except:
        return None

## Actually gather the information using apply statements
#### writing updated dataframes to temp file in between calls
#### tqdm makes this a lot less painful, because can see progress
See:

http://stackoverflow.com/questions/40476680/how-to-use-tqdm-with-pandas-in-a-jupyter-notebook

http://stackoverflow.com/questions/18603270/progress-indicator-during-pandas-operations-python

https://pypi.python.org/pypi/tqdm

In [17]:
tqdm_notebook().pandas()
both['SIC'] = both.progress_apply(lambda row: lookup_sic_code(row['Symbol']), axis=1)
both.to_csv("tmp/TEMP_BOTH.csv")

In [30]:
tqdm_notebook().pandas()
both['bloomberg'] = both.progress_apply(lambda row: lookup_bloomberg_profile(row['Symbol']), axis=1)
both.to_csv("tmp/TEMP_BOTH.csv")

In [31]:
tqdm_notebook().pandas()
both['google'] = both.progress_apply(lambda row: lookup_google_profile(row['Symbol'], row["Market"]), axis=1)
both.to_csv("tmp/TEMP_BOTH.csv")

In [32]:
tqdm_notebook().pandas()
both['reuters'] = both.progress_apply(lambda row: lookup_reuters_profile(row['Symbol']), axis=1)
both.to_csv("tmp/TEMP_BOTH.csv")

## Joining to OpenSecrets categories/sectors/industries
#### Using tables from:
* https://groups.google.com/forum/#!topic/opensecrets-open-data/nXYSeFrtwxk
* https://www.opensecrets.org/downloads/crp/CRP_Categories.txt (with its header removed)

In [139]:
sic2crpcat = pd.read_csv("../data/SIC/conversion/SIC2CRPCat.csv")
crp_cats = pd.read_csv("../data/SIC/CRP_Categories_no_head.txt", sep = "\t")

In [141]:
both_joined = pd.merge(left = both, right = sic2crpcat, left_on='SIC', right_on='SICcode', how='left')
both_joined = pd.merge(left = both_joined, right = crp_cats, on="Catcode", how='left')

In [145]:
both_joined_raw

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector_x,industry,Summary Quote,Unnamed: 8,Market,...,SICcode,Catcode,Category,SICname,MultSIC,Catname,Catorder,Industry,Sector_y,Sector Long
0,PIH,"1347 Property Insurance Holdings, Inc.",7.5,$44.68M,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN,NASDAQ,...,6331,F3400,Property & casualty insurance,"Fire, marine, and casualty insurance",NaN,Property & casualty insurance,F09,Insurance,Finance/Insur/RealEst,"Finance, Insurance & Real Estate"
1,TURN,180 Degree Capital Corp.,1.43,$44.19M,n/a,Finance,Finance/Investors Services,http://www.nasdaq.com/symbol/turn,NaN,NASDAQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FLWS,"1-800 FLOWERS.COM, Inc.",10,$655.69M,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN,NASDAQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FCCY,1st Constitution Bancorp (NJ),17.85,$143.29M,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN,NASDAQ,...,6035,F1200,Savings banks & Savings and Loans,Federal savings institutions,X,Savings banks & Savings and Loans,F04,Savings & Loans,Finance/Insur/RealEst,"Finance, Insurance & Real Estate"
4,SRCE,1st Source Corporation,44.55,$1.2B,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN,NASDAQ,...,6022,F1100,Commercial banks & bank holding companies,State commercial banks,X,Commercial banks & bank holding companies,F03,Commercial Banks,Finance/Insur/RealEst,"Finance, Insurance & Real Estate"
5,VNET,"21Vianet Group, Inc.",5.55,$628.87M,2011,Technology,"Computer Software: Programming, Data Processing",http://www.nasdaq.com/symbol/vnet,NaN,NASDAQ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,TWOU,"2U, Inc.",41.72,$1.97B,2014,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/twou,NaN,NASDAQ,...,7372,C5120,Computer software,Prepackaged software,NaN,Computer software,B12,Electronics Mfg & Equip,Communic/Electronics,Communications/Electronics
7,JOBS,"51job, Inc.",38.07,$2.29B,2004,Technology,Diversified Commercial Services,http://www.nasdaq.com/symbol/jobs,NaN,NASDAQ,...,7361,G5250,Employment agencies,Employment agencies,X,Employment agencies,N05,Business Services,Misc Business,Misc Business
8,CAFD,8point3 Energy Partners LP,12.22,$966.32M,2015,Public Utilities,Electric Utilities: Central,http://www.nasdaq.com/symbol/cafd,NaN,NASDAQ,...,4911,E1600,Electric Power utilities,Electric services,NaN,Electric Power utilities,E08,Electric Utilities,Energy/Nat Resource,Energy & Natural Resources
9,EGHT,8x8 Inc,14.95,$1.36B,n/a,Public Utilities,Telecommunications Equipment,http://www.nasdaq.com/symbol/eght,NaN,NASDAQ,...,4813,C4100,Telephone utilities,"Telephone communications, exc radio",X,Telephone utilities,B08,Telephone Utilities,Communic/Electronics,Communications/Electronics


In [144]:
both_joined_raw = both_joined
both_joined_raw.to_csv('tmp/TEMP_both_joined_raw.csv')

## Clean up the data a bit

In [146]:
# subset columns
both_joined_processed = both_joined.loc[:,['Symbol', 'Name', 'Market', 'SIC',
                                 'bloomberg','google','reuters','Catcode', 'Catname','Industry','Sector Long']]
# remove whitespace, and replace empty remaining strings with NaN (for whatever reason, 
# replacing with None fills with the value of row above)
both_joined_processed.bloomberg = both_joined_processed.bloomberg.str.strip().replace("",np.nan)
both_joined_processed.google = both_joined_processed.google.str.strip().replace("",np.nan)
both_joined_processed.reuters = both_joined_processed.reuters.str.strip().replace("",np.nan)
# remove rows that don't have SIC/Catcode/Catname (making sure joined properly)
# and rows that don't have at least one text column
both_joined_processed = both_joined_processed[(~(both_joined_processed.Catname.isnull())) & 
                                              (~(both_joined_processed.Catcode.isnull())) & 
                                              (~(both_joined_processed.SIC.isnull())) & 
                                              ((~(both_joined_processed.bloomberg.isnull())) | 
                                               (~(both_joined_processed.google.isnull())) | 
                                               (~(both_joined_processed.reuters.isnull()))
                                              )]

## Write data

In [148]:
both_joined_processed.to_csv('../data/lincoln/companies_catcodes_text.csv',index=False)

In [149]:
len(both_joined_processed)

3623

In [ ]:
# # turns out this is a bit tricky
# def lookup_yahoo_profile(symbol):
#     return None
# #     s = Share(symbol)
# #     print(s.get_info())
# #     url = "http://finance.yahoo.com/quote/" + symbol + "/profile"
# #     r = requests.get(url)
# #     soup = bs(r.text,"lxml")
# #     description = soup.findAll('section', {'aria-label' : "Main Column"})[0].child
# #     print(description)